In [ ]:
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

In [60]:
import pm4py
import numpy as np
import pandas as pd
import datetime as dt

from copy import deepcopy
from pm4py.util.benchmarking import benchmark_event_log_from_configs
from pm4py.algo.discovery.dcr_discover.algorithm import Variants

# Mine with SpT-DisCoveR

# Sepsis

In [ ]:
event_log_file = '/home/vco/Datasets/Sepsis Cases - Event Log.xes'
dcr_title = 'Sepsis Cases'
result_file_prefix = 'sepsis'
config = {
    'findAdditionalConditions' : True,
    'inBetweenRels' : True,
    'timed' : False,
    'discardSelfInPredecessors': True,
    'usePredecessors': False
}
sepsis_reference_dcr, sepsis_event_log, sepsis_sp_dcr, sepsis_sp_log = \
    benchmark_event_log_from_configs(event_log_file,result_file_prefix,dcr_title,config)

# BPIC 2017

In [ ]:
timed = True

event_log_file = '/home/vco/Datasets/BPI Challenge 2017 - Offer log.xes'
dcr_title = 'BPIC2017'
result_file_prefix = 'bpic2017'
configs = [{
    'inBetweenRels': True,
    'timed': timed,
    'variant': Variants.DCR_SUBPROCESS_ME
}]
bpic12_dcr_graphs = benchmark_event_log_from_configs(event_log_file, result_file_prefix, dcr_title, configs)

In [ ]:
event_log_file = '/home/vco/Datasets/BPI Challenge 2017 - Offer log.xes'
dcr_title = 'BPIC2017'
result_file_prefix = 'bpic2017'
config = {
    'findAdditionalConditions' : True,
    'inBetweenRels' : True,
    'timed' : False,
    'discardSelfInPredecessors': True,
    'usePredecessors': False
}
bpic17_reference_dcr, bpic17_event_log, bpic17_sp_dcr, bpic17_sp_log =\
    benchmark_event_log_from_configs(event_log_file,result_file_prefix,dcr_title,config)

# Road Traffic

In [ ]:
event_log_file = '/home/vco/Datasets/12683249/Road_Traffic_Fine_Management_Process.xes'
dcr_title = 'Traffic Fine'
result_file_prefix = 'traffic_fine'
config = {
    'findAdditionalConditions' : True,
    'inBetweenRels' : True,
    'timed' : False,
    'discardSelfInPredecessors': True,
    'usePredecessors': False
}
rtfmp_reference_dcr, rtfmp_event_log, rtfmp_sp_dcr, rtfmp_sp_log =\
benchmark_event_log_from_configs(event_log_file,result_file_prefix,dcr_title,config)

# BPIC2017: Something Thomas asked about

In [ ]:
df = pm4py.convert_to_dataframe(bpic17_event_log)
#TODO: check if Created exists but afterwards there is no Refused/Accepted/Cancelled (OR)
df[df['concept:name']=='0_Created']

# Visualize as histograms

In [ ]:
from pm4py.algo.discovery.dcr_discover import time_mining
from pm4py.visualization.timings import visualizer as t_vis

timings = time_mining.get_timing_values(rtfmp_reference_dcr, rtfmp_event_log)
clear_output(wait=True)
t_vis.apply(timings, xmin=0, xmax=500)

# Playground stuff

In [ ]:
#test trace to test timings
cids = [1,1,1,1,1,1,
        2,2,2,2,2,
        3,3,3,3,3,3,3,3,3,3,
        4,4,4,4,4,4,4,4,4,4,4,4,4,4,4]
events = ['B','B','C','D','E','F',
          'A', 'B', 'E', 'F' ,'C',
          'A', 'B', 'F', 'C', 'B', 'C', 'B', 'E', 'F', 'C',
          'B','B','B','Z','B','C','B','A','C','Z','C','C','B','B','C']
timestamps = [pd.to_datetime('2021-8-02'),pd.to_datetime('2021-10-02'),pd.to_datetime('2021-10-03'),
                      pd.to_datetime('2021-10-04'),pd.to_datetime('2021-10-05'),pd.to_datetime('2021-10-06'),
                     pd.to_datetime('2021-10-01'),pd.to_datetime('2021-10-02'),pd.to_datetime('2021-10-03'),
                      pd.to_datetime('2021-10-04'),pd.to_datetime('2021-10-05'),
                     pd.to_datetime('2021-10-01'),pd.to_datetime('2021-10-02'),pd.to_datetime('2021-10-03'),
                      pd.to_datetime('2021-10-04'),pd.to_datetime('2021-10-05'),pd.to_datetime('2021-10-06'),pd.to_datetime('2021-10-07'),
                      pd.to_datetime('2021-10-08'),pd.to_datetime('2021-10-09'),pd.to_datetime('2021-10-10'),
                     pd.to_datetime('2021-9-01'),pd.to_datetime('2021-10-01'),pd.to_datetime('2021-10-03'),
                      pd.to_datetime('2021-10-04'),pd.to_datetime('2021-10-05'),pd.to_datetime('2021-10-06'),
                      pd.to_datetime('2021-10-07'),pd.to_datetime('2021-10-08'),pd.to_datetime('2021-10-09'),
                      pd.to_datetime('2021-10-10'),pd.to_datetime('2021-10-11'),pd.to_datetime('2025-10-11'),
                      pd.to_datetime('2025-10-12'),pd.to_datetime('2025-10-13'),pd.to_datetime('2025-10-14')]
test_log_dict = {
    'case:concept:name':cids,
    'concept:name':events,
    'time:timestamp':timestamps
}

test_tid = {
    'CONDITION': [['B','C']],
    'RESPONSE': [['B','C']]
}

test_log_df = pd.DataFrame(test_log_dict)
test_event_log = deepcopy(test_log_df)

In [63]:
#test traces for subproceses
dict_keys = ['case:concept:name','concept:name','time:timestamp']

def create_test_traces(ordered_event_matrix):
    start_date = dt.date(2023,1,1)
    case_id = 1
    test_el = []
    for trace in ordered_event_matrix:
        date = start_date
        for event in trace:
            test_el.append({dict_keys[0]:str(case_id),dict_keys[1]:str(event),dict_keys[2]:date})
            date = date + dt.timedelta(days=np.random.randint(1, 10)) # TODO: there could be a random sample from a distribution here
        case_id = case_id + 1
    df_test = pd.DataFrame(test_el,columns=['case:concept:name','concept:name','time:timestamp'],dtype=str)
    df_test['time:timestamp'] = pd.to_datetime(df_test['time:timestamp'])
    return pm4py.convert_to_event_log(df_test)

In [ ]:
A,B,C,D,E,F = 'A','B','C','D','E','F'
event_log = create_test_traces([[A,C,C,B,D],[A,B,C,C,D],[A,E,F,D],[A,F,E,D],[A,B,C,D,A,E,F,D],[A,C,B,C,D,A,F,E,D]])

In [64]:
A,B,C,D = 'I1','I2','F1','F2'
running_example_trace = create_test_traces([[A,C],[A,D],[B,C],[B,D]])
tt = pm4py.convert_to_event_log(running_example_trace)
pm4py.write_xes(tt,file_path='/home/vco/Downloads/running_example_trace.xes')

/home/vco/Projects/pm4py-dcr/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


2023-01-08
2023-01-11
2023-01-08
2023-01-15
2023-01-02
2023-01-05
2023-01-04
2023-01-10


exporting log, completed traces :: 100%|██████████| 4/4 [00:00<00:00, 23109.11it/s]


In [65]:
running_example_trace

[{'attributes': {'concept:name': '1'}, 'events': [{'concept:name': 'I1', 'time:timestamp': Timestamp('2023-01-01 00:00:00')}, '..', {'concept:name': 'F1', 'time:timestamp': Timestamp('2023-01-08 00:00:00')}]}, '....', {'attributes': {'concept:name': '4'}, 'events': [{'concept:name': 'I2', 'time:timestamp': Timestamp('2023-01-01 00:00:00')}, '..', {'concept:name': 'F2', 'time:timestamp': Timestamp('2023-01-04 00:00:00')}]}]

In [67]:
configs = [{
            'inBetweenRels': True,
            'timed': True,
            'pending': True,
            'variant': Variants.DCR_BASIC
        },{
            'inBetweenRels': False,
            'timed': True,
            'pending': True,
            'variant': Variants.DCR_SUBPROCESS_ME
        }
]
dcrs = benchmark_event_log_from_configs(tt,dcr_title="running_example",result_file_prefix="running_example",configs=configs)

[i] Started with config: {'inBetweenRels': True, 'timed': True, 'pending': True, 'variant': <Variants.DCR_BASIC: <module 'pm4py.algo.discovery.dcr_discover.variants.discover_basic' from '/home/vco/Projects/pm4py-dcr/pm4py/algo/discovery/dcr_discover/variants/discover_basic.py'>>}
[i] Mining with basic DisCoveR
[!] Model saved in models/running_example_T_config0.xml
[i] Started with config: {'inBetweenRels': False, 'timed': True, 'pending': True, 'variant': <Variants.DCR_SUBPROCESS_ME: <module 'pm4py.algo.discovery.dcr_discover.variants.discover_subprocess_mutual_exclusion' from '/home/vco/Projects/pm4py-dcr/pm4py/algo/discovery/dcr_discover/variants/discover_subprocess_mutual_exclusion.py'>>}
[i] Mining with Sp-DisCoveR (ME)
[subprocess] {'F2', 'F1'}
[subprocess] {'I2', 'I1'}
[subprocess] {'I2', 'I1'}
[subprocess] {'F2', 'F1'}
[!] Model saved in models/running_example_T_config1.xml


/home/vco/Projects/pm4py-dcr/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")
/home/vco/Projects/pm4py-dcr/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [68]:
dcrs[0]

{'events': {'F1', 'F2', 'I1', 'I2'},
 'conditionsFor': {'F2': {'I1', 'I2'},
  'I2': set(),
  'I1': set(),
  'F1': {'I1', 'I2'}},
 'milestonesFor': {'F2': set(), 'I2': set(), 'I1': set(), 'F1': set()},
 'responseTo': {'F2': set(), 'I2': set(), 'I1': set(), 'F1': set()},
 'includesTo': {'F2': set(), 'I2': set(), 'I1': set(), 'F1': set()},
 'excludesTo': {'F2': {'F1', 'F2'},
  'I2': {'I1', 'I2'},
  'I1': {'I1', 'I2'},
  'F1': {'F1', 'F2'}},
 'marking': {'executed': set(),
  'included': {'F1', 'F2', 'I1', 'I2'},
  'pending': {'F1', 'F2', 'I1', 'I2'}},
 'conditionsForDelays': {'F1': {'I1': Timedelta('7 days 00:00:00'),
   'I2': Timedelta('1 days 00:00:00')},
  'F2': {'I1': Timedelta('7 days 00:00:00'),
   'I2': Timedelta('3 days 00:00:00')}},
 'responseToDeadlines': {}}

In [69]:
dcrs[1]

{'events': {'F1', 'F2', 'I1', 'I2', 'S0', 'S1'},
 'conditionsFor': {'S1': set(), 'S0': {'S1'}},
 'milestonesFor': {},
 'responseTo': {'S1': {'S0'}, 'S0': set()},
 'includesTo': {'S1': {'S0'}, 'S0': set()},
 'excludesTo': {'S1': {'S1'}, 'S0': {'S0'}},
 'marking': {'executed': set(),
  'included': {'F1', 'F2', 'I1', 'I2', 'S0', 'S1'},
  'pending': {'S0', 'S1'}},
 'conditionsForDelays': {'S0': {'S1': Timedelta('1 days 00:00:00')}},
 'responseToDeadlines': {'S1': {'S0': Timedelta('7 days 00:00:00')}},
 'subprocesses': {'S0': {'F1', 'F2'}, 'S1': {'I1', 'I2'}}}

In [70]:
import networkx as nx

In [76]:
df = pm4py.convert_to_dataframe(running_example_trace)

In [82]:
for t in running_example_trace:
    trace_to_print = ''
    for e in t:
        trace_to_print += e['concept:name']+', '
    print(trace_to_print[:-2])

I1, F1
I1, F2
I2, F1
I2, F2


In [83]:
df

,concept:name,time:timestamp,case:concept:name
0,I1,2023-01-01 00:00:00+00:00,1
1,F1,2023-01-08 00:00:00+00:00,1
2,I1,2023-01-01 00:00:00+00:00,2
3,F2,2023-01-08 00:00:00+00:00,2
4,I2,2023-01-01 00:00:00+00:00,3
5,F1,2023-01-02 00:00:00+00:00,3
6,I2,2023-01-01 00:00:00+00:00,4
7,F2,2023-01-04 00:00:00+00:00,4
